# ĐỒ ÁN CUỐI KỲ - NHẬP MÔN KHOA HỌC DỮ LIỆU
STT nhóm: 17

Thành viên:
- 18120066 - Bùi Đoàn Hữu Nhân
- 18120097 - Đinh Hữu Phúc Trung

##  Đề tài: Dự đoán giá xe ô tô

---

# I - Thu thập dữ liệu
&#9889; **Lưu ý:**

Để có thể chạy file notebook này ta cần phải:
- Install thư viện Selenium.
- Tải file thực thi thích hợp cho từng máy va từng phiên bản của trình duyệt mà ta sử dụng. 
    - Nếu máy dùng trình duyệt Google Chrome: bạn cần download driver [tại đây](https://chromedriver.chromium.org/downloads)
    - Nếu máy dùng trình duyệt Microsoft Edge (Chromium): bạn cần download driver [tại đây](https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/)
    
Trong đồ án, chúng em sử dụng trình duyệt Microsoft Edge (Chromium) nên đã tải sẵn file thực thi và lưu vào chung thư mục với file notebook này.

Dưới đây là code thu thập dữ liệu với số lượng ít (dùng để kiểm tra và chạy thử), còn việc thu thập dữ liệu để sử dụng cho bài làm có số lượng lớn hơn rất nhiều đã được thực hiện trước vì việc thu thập tốn khá nhiều thời gian (run time lớn).

---

## Import

In [5]:
import requests
import urllib.robotparser
import numpy as np
import pandas as pd
import re
import time
import datetime

from bs4 import BeautifulSoup,SoupStrainer
from selenium import webdriver

---

## Kiểm tra trước khi crawl dữ liệu

In [6]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.cars-data.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.cars-data.com/en/all-cars/page1.html')

True

In [7]:
rp.can_fetch('*', 'https://www.cars-data.com/en/abarth-500-1-4-16v-t-jet-specs/1/tech')

True

---

## Bắt đầu crawl dữ liệu

Đây là trang web sử dụng javascript, nên ta không thể dùng phương pháp parser của request được. Ta cần sử dụng một thư viện khác để có thể thu thập dữ liệu của trang web là Selenium.

### Mở trình duyệt mà ta cần để lấy dữ liệu trang web

Với mỗi trình duyệt khác nhau thì câu lệnh cũng khác nhau. Ở đây, ta đang sử dụng trình duyệt Edge nên câu lệnh được thực hiện như sau.

(`executable_path` là đường dẫn đến file thực thi)

In [8]:
browser = webdriver.Edge(executable_path='msedgedriver.exe')

### Thu thập link chứa thông tin của mỗi xe

- Nếu link nhận được là link của một chiếc xe cụ thể (không có option) thì được thêm trực tiếp vào danh sách info_cars (ta có thể nhận thấy, thông tin chi tiết của xe sẽ là đường dẫn của xe có đi đến "/tech")
- Nếu link nhận được là link của một chiếc xe có các option khác, thì link này sẽ được gán vào danh sách links để bước sau ta sẽ xử lí trường hợp này.

*Thay đổi `num_pages` để thay đổi số lượng trang mà ta cần thu thập, và các page được lấy theo thứ tự giảm dần nhằm mục đích lấy được nhiều xe có năm ra mắt là mới nhất.*

In [9]:
keyword = 97   # 97 là số trang tối đa
numPage = 1   # numPage là số lượng trang ta muốn lấy dữ liệu
links = list()
info_cars = list()
while (keyword - (97 - numPage) > 0):
    browser.get(f'https://www.cars-data.com/en/all-cars/page{keyword}.html')
    html_text = browser.page_source

    tree = BeautifulSoup(html_text, 'lxml')
    cars_box = tree.find_all('div',{'class':'col-4'})
    for car in cars_box[0:-3]:
        link = car.find('a')['href']
        if (link.find('specs') == -1):
            links.append(link)
        else:
            info_cars.append(link + '/tech')
    keyword-=1

Tiếp theo ta sẽ lấy luôn các option của từng xe mà có trong danh sách  `links ` (thông tin chi tiết của xe sẽ là đường dẫn của xe có đi đến "/tech").

In [10]:
for link in links:
    browser.get(link)
    html_text = browser.page_source

    tree = BeautifulSoup(html_text, 'lxml')
    options = tree.find_all('div',{'class':'col-6'})
    for option in options:
        info = option.find('a')['href'] + '/tech'
        info_cars.append(info)

Như vậy ta đã thu thập đủ link chứa thông tin của các xe.

In [11]:
len(info_cars)

29

### Xử lý và lấy thông tin chi tiết của mỗi xe
- Do thông tin của xe (không lấy xe điện) nằm trong một bảng cố định. Ở bước sau, khi ta thấy thông tin đó thì trong danh sách nhận được, các dòng chẵn sẽ là các đề mục, và dòng lẻ (+1) sẽ là các giá trị của đề mục đó. Do không phải ta sẽ lấy hết tất cả các thông số của xe mà ta chỉ lấy những thông tin có thể cần thiết cho việc huấn luyện sau này, nên ta cần phải biết được những vị trí cần lấy trong danh sách thông tin đó. Từ đó là sẽ có được các index cho dictionary  `label `. Các giá trị của index chính là các tên cột của dataframe sau này.
- Vì vậy ta sẽ dùng dictionary để có thể tận dụng được index cho việc lấy thông tin và giá trị cho việc tạo tên cột dataframe.
- Tạo dataframe, ta sẽ thêm hai cột đó là Name (tên xe. VD: Toyota Camry,...), là Brand (thương hiệu xe. VD: Toyota,...)

In [12]:
label = {0:'Price',6:'Body',8:'Transmission',10:'Number Of Seats',12:'Segment',14:'Introduction',18:'Drive',20:'Drive System',22:'Fuel',34:'Valves Per Cylinder',34:'Cylinder Capacity',46:'Max Power Hp',50:'Max Torque',54:'Fuel System',56:'Valve Actuation',58:'Turbo',62:'Fuel Tank',64:'Top Speed',78:'Energy Label',108:'Front Stabilizer',110:'Rear Stabilizer'}

In [33]:
df = pd.DataFrame(columns=['Name','Brand']+list(label.values()))

- Ở bước này, ta cần tách Brand và Name từ đường link của xe, đồng thời sẽ lấy thông tin chi tiết của mỗi xe theo cách đã trình bày ở trên.
- Ta dùng Try Except ở bước này nhằm bắt được các lỗi sai trong quá trình lấy thông tin của xe (thông tin của xe điện ít hơn xe khác,...)

*Sau khi thực hiện xong bước này, ta đã có được dataframe dùng để huấn luyện*

In [34]:
i=0
for info in info_cars:
    browser.get(info)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text,'lxml',parse_only=SoupStrainer('td'))
    info_detail = tree.find_all('td',attrs={'class': re.compile(r"^col-6$")})
    if (len(info_detail) == 146):
        list_tmp = list()
        i+=1
        try:
            name_car = info[29:]
            name_car = name_car[:name_car.find('-specs')].replace('-',' ').upper()
            brand = name_car[:name_car.find(' ')]
            list_tmp.append(name_car)
            list_tmp.append(brand)
            for j in label:
                list_tmp.append(info_detail[j+1].text)
            df.loc[i] = list_tmp
        except:
            i-=1

Hoàn thành việc thu thập dữ liệu.

In [16]:
df.dtypes

Name                 object
Brand                object
Price                object
Body                 object
Transmission         object
Number Of Seats      object
Segment              object
Introduction         object
Drive                object
Drive System         object
Fuel                 object
Cylinder Capacity    object
Max Power Hp         object
Max Torque           object
Fuel System          object
Valve Actuation      object
Turbo                object
Fuel Tank            object
Top Speed            object
Energy Label         object
Front Stabilizer     object
Rear Stabilizer      object
dtype: object

Số dòng dữ liệu ở đây ít hơn số link thu thập được do có một số xe ôtô chạy bằng điện và ta bỏ qua loại xe này vì cấu tạo của nó khác so với những xe khác.

---

## Xử lý dữ liệu thô
Dữ liệu thu thập về đang còn ở mức rất thô, ta sẽ tiến hành xử lý từng cột để định dạng lại dữ liệu này trước khi thực hiện Tiền xử lý và Mô hình hóa dữ liệu.

In [ ]:
df = pd.read_csv('data_root.csv',index_col = 0)

In [140]:
# Xử lí cột "Price" - thực hiện tách đơn vị và chuyển kiểu dữ liệu thành dạng số
df['Price'] = pd.to_numeric(df['Price'].str.extract(r'([\d.]+)', expand = False), errors='coerce')*1000

In [141]:
# Xử lí cột "Body" - cột này chứa 2 thông tin là số cửa và loại body
# Ta thực hiện tách ra thành hai cột: Num_door và Body
Num_doors = list()
Body = list()
for i in df['Body']:
    lc = i.find(',')
    Num_doors.append(i[:lc-6])
    Body.append(i[lc+1:])
df['Body'] = Body
df['Num_doors'] = pd.to_numeric(pd.Series(Num_doors), errors='coerce')

In [142]:
# Xử lí cột "Transmission" - chứa số cấp bậc của hộp số và loại hộp số
# Tách ra làm 2 cột là Transmission và Dt_Transmission
Transmission = list()
Dt_Transmission = list()
for i in df['Transmission']:
    lc = i.find('speed')
    Transmission.append(i[:lc-1])
    Dt_Transmission.append(i[lc+6:])
df['Dt_Transmission'] = Dt_Transmission
df['Transmission'] = pd.to_numeric(pd.Series(Transmission), errors='coerce')

In [143]:
# Chuyển dữ liệu "Number Of Seats" về dạng số nếu có dạng a-b thì lấy b
df['Number Of Seats'] = pd.to_numeric([x if x.find('-')==-1 else x[x.find('-')+1:] for x in df['Number Of Seats']],
                                      errors='coerce')

In [144]:
# Ở cột "Segment" tách lấy ký tự ký hiệu cho loại kích cỡ xe
df['Segment'] = df['Segment'].str.extract(r'(\w)', expand = False)

In [145]:
# Tách lấy năm trong "Introduction"
df['Introduction'] = pd.to_numeric(df['Introduction'].str.extract(r'(\d+)', expand = False), errors='coerce')

In [146]:
# Chuyển cột "Cylinder Capacity" về kiểu dữ liệu số 
df['Cylinder Capacity'] = pd.to_numeric(df['Cylinder Capacity'], errors='coerce')

In [147]:
# Chuyển cột "Max Power Hp" về kiểu dữ liệu số 
df['Max Power Hp'] = pd.to_numeric(df['Max Power Hp'], errors='coerce')

In [148]:
# Tách "Max Torque" ra khỏi đơn vị và chuyền về dạng số 
df['Max Torque'] = pd.to_numeric(df['Max Torque'].str.extract(r'(\d+)', expand = False), errors='coerce')

In [149]:
# Xử lí cột "Valve Actuation"
df['Valve Actuation'] = [np.nan if x == 'n.b.' else x for x in df['Valve Actuation']]

In [150]:
# Xử lí cột "Turbo" về dịnh dạng Yes/No
df['Turbo'] = df['Turbo'].str.extract(r'(\w+)', expand = False)

In [151]:
# Tách "Fuel Tanke" ra khỏi đơn vị 
df['Fuel Tank'] = df['Fuel Tank'].str.extract(r'(\d+)', expand = False)

In [152]:
# Tách "Top Speed" ra khỏi đơn vị và chuyển về dạng số
df['Top Speed'] = pd.to_numeric(df['Top Speed'].str.extract(r'(\d+)', expand = False), errors='coerce')

In [1]:
df.head()

NameError: name 'df' is not defined

## Xuất dữ liệu ra file
Cuối cùng, ta xuất dữ liệu thu thập được ra file "data.csv" để sử dụng trong phần tiếp theo của đồ án.

In [ ]:
df.to_csv('cars_data.csv')